# Sudoku-7-Norvig-Python : Solveur de Peter Norvig (Python)**Navigation** : [<< AIMA CSP](Sudoku-6-AIMA-CSP-Csharp.ipynb) | [Index](README.md) | [Human Strategies >>](Sudoku-8-HumanStrategies-Csharp.ipynb)## Objectifs d'apprentissageA la fin de ce notebook, vous saurez :1. **Comprendre** l'approche de propagation de contraintes de Peter Norvig2. **Implementer** les techniques d'elimination et de naked singles3. **Utiliser** le backtracking avec l'heuristique MRV4. **Resoudre** n'importe quel Sudoku en moins d'une seconde**Duree estimee** : ~15 min | **Prerequis** : [Sudoku-0 Environment](Sudoku-0-Environment-Csharp.ipynb)---Ce notebook implemente le celebre solveur de **Peter Norvig** (2006), considere comme l'une des implementations les plus elegantes et efficaces pour le Sudoku. Le code original est disponible sur [norvig.com/sudoku.html](http://norvig.com/sudoku.html).

In [ ]:
# Importsimport timefrom typing import Dict, Set, Tuple, Optional, Listprint('Solveur Sudoku - Approche Norvig')

## Introduction : L'approche NorvigPeter Norvig a propose en 2006 une solution elegante en ~80 lignes de Python qui resout n'importe quel Sudoku en moins d'une seconde.### Concepts cles- **Digits** : les chiffres '1' a '9'- **Carres (Squares)** : les 81 cases nommees 'A1' a 'I9'- **Unites (Units)** : chaque ligne, colonne ou bloc 3x3 (27 unites au total)- **Pairs (Peers)** : les 20 cases qui partagent une unite avec une case donnee### Strategie1. **Propagation** : Reduire les domaines par elimination2. **Elimination** : Si une case a une valeur, eliminer cette valeur des pairs3. **Naked Singles** : Si une case n'a qu'une valeur possible, l'assigner4. **Backtracking** : Si bloque, faire un retour arriere intelligent (MRV)

In [ ]:
# Definition des constantesdigits   = '123456789'rows     = 'ABCDEFGHI'cols     = digitssquares  = [r + c for r in rows for c in cols]# Unites : lignes, colonnes, blocsunitlist = ([['A'+c, 'B'+c, 'C'+c, 'D'+c, 'E'+c, 'F'+c, 'G'+c, 'H'+c, 'I'+c] for c in cols] +            [[r+'1', r+'2', r+'3', r+'4', r+'5', r+'6', r+'7', r+'8', r+'9'] for r in rows] +            [[r+c for r in rs for c in cs] for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])# Dictionnaire : square -> liste des 3 unitesunits = {s: [u for u in unitlist if s in u] for s in squares}# Dictionnaire : square -> ensemble des 20 pairspeers = {s: set(sum(units[s], [])) - {s} for s in squares}print(f'Carres: {len(squares)}')print(f'Unites: {len(unitlist)}')print(f'Pairs de A1: {len(peers["A1"])}')print(f'Unites de A1: {units["A1"]}')

## Parsing d'une grilleUne grille est representee comme un dictionnaire `{square: digits}` ou `digits` est la chaine des valeurs possibles.

In [ ]:
def parse_grid(grid: str) -> Optional[Dict[str, str]]:    """Convertit une grille string en dictionnaire {square: digits}."""    values = {s: digits for s in squares}  # Toutes les valeurs possibles    for s, d in grid_values(grid).items():        if d in digits and not assign(values, s, d):            return None  # Contradiction    return valuesdef grid_values(grid: str) -> Dict[str, str]:    """Convertit une grille en dictionnaire {square: char}."""    chars = [c for c in grid if c in digits or c in '0.']    assert len(chars) == 81    return dict(zip(squares, chars))# Testtest_grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'print(f'Grille test: {len(test_grid)} caracteres')

## Propagation des contraintesLes deux operations fondamentales :1. **assign(values, square, digit)** : Assigner une valeur et propager2. **eliminate(values, square, digit)** : Eliminer une valeur et propager

In [ ]:
def assign(values: Dict[str, str], s: str, d: str) -> bool:    """Assigne d a s en eliminant toutes les autres valeurs."""    other_values = values[s].replace(d, '')    if all(eliminate(values, s, d2) for d2 in other_values):        return values    else:        return False  # Contradictiondef eliminate(values: Dict[str, str], s: str, d: str) -> bool:    """Elimine d de values[s]; propage les consequences."""    if d not in values[s]:        return True  # Deja elimine    values[s] = values[s].replace(d, '')        # (1) Si une case n'a qu'une valeur, l'eliminer des pairs    if len(values[s]) == 0:        return False  # Contradiction: plus de valeurs    elif len(values[s]) == 1:        d2 = values[s]        if not all(eliminate(values, s2, d2) for s2 in peers[s]):            return False        # (2) Si une unite n'a qu'une place pour d, l'y assigner    for u in units[s]:        dplaces = [s2 for s2 in u if d in values[s2]]        if len(dplaces) == 0:            return False  # Contradiction: pas de place pour d        elif len(dplaces) == 1:            if not assign(values, dplaces[0], d):                return False    return True

## Resolution avec BacktrackingSi la propagation ne suffit pas, on utilise le backtracking avec l'heuristique MRV (Minimum Remaining Values) : choisir la case avec le moins de valeurs possibles.

In [ ]:
def solve(grid: str) -> Optional[Dict[str, str]]:    """Resout une grille Sudoku."""    return search(parse_grid(grid))def search(values: Optional[Dict[str, str]]) -> Optional[Dict[str, str]]:    """Backtracking avec heuristique MRV."""    if values is None:        return None  # Echec    if all(len(values[s]) == 1 for s in squares):        return values  # Resolu !        # Choisir la case avec le moins de valeurs (MRV)    n, s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)        # Essayer chaque valeur    for d in values[s]:        result = search(assign(values.copy(), s, d))        if result:            return result    return None

## Affichage des resultats

In [ ]:
def display(values: Dict[str, str]) -> str:    """Affiche une grille."""    width = 1 + max(len(values[s]) for s in squares)    line = '+'.join(['-' * (width * 3)] * 3)    result = []    for r in rows:        result.append(''.join(values[r + c].center(width) + ('|' if c in '36' else '')                                for c in cols))        if r in 'CF':            result.append(line)    return '\n'.join(result)# Test avec une grille facileeasy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'print('Puzzle:')values = parse_grid(easy)print(display(values))print('\nResolution...')start = time.time()solution = solve(easy)elapsed = (time.time() - start) * 1000if solution:    print(f'\nResolu en {elapsed:.2f} ms')    print(display(solution))else:    print('Pas de solution')

## BenchmarkTestons le solveur sur des puzzles de differentes difficultes.

In [ ]:
from pathlib import PathPUZZLES_DIR = Path(r'D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku\Puzzles')def load_puzzles(filepath: str, max_puzzles: int = None):    puzzles = []    with open(filepath, 'r') as f:        for line in f:            line = line.strip()            if len(line) >= 81:                puzzles.append(line[:81])                if max_puzzles and len(puzzles) >= max_puzzles:                    break    return puzzlesdef benchmark(puzzles: list, name: str):    print(f'\nBenchmark: {name} ({len(puzzles)} puzzles)')        start = time.time()    solved = 0    for puzzle in puzzles:        if solve(puzzle):            solved += 1    elapsed = (time.time() - start) * 1000        print(f'  Resolus: {solved}/{len(puzzles)}')    print(f'  Temps total: {elapsed:.2f} ms')    print(f'  Temps moyen: {elapsed/len(puzzles):.2f} ms/puzzle')# Testseasy = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=20)hard = load_puzzles(str(PUZZLES_DIR / 'Sudoku_hardest.txt'))benchmark(easy, 'Puzzles Faciles')benchmark(hard, 'Puzzles Difficiles')

## Resume### Points cles du solveur Norvig1. **Propagation** : Reduit les domaines avant la recherche2. **Elimination** : Si une case a une valeur, l'eliminer des pairs3. **Naked Singles** : Si une case n'a qu'une valeur possible, l'assigner4. **Hidden Singles** : Si une unite n'a qu'une place pour une valeur, l'y mettre5. **MRV** : Choisir la case avec le moins de valeurs pour le backtracking### Performance| Type | Temps moyen ||------|-------------|| Facile | <1 ms || Difficile | <10 ms || Top95 | ~1-5 ms |### Pourquoi ca marche ?- La **propagation** elimine 95%+ des cas sans backtracking- L'heuristique **MRV** minimise les retours arrieres- La structure de donnees (dict de strings) est simple et efficace---**Navigation** : [<< AIMA CSP](Sudoku-6-AIMA-CSP-Csharp.ipynb) | [Index](README.md) | [Human Strategies >>](Sudoku-8-HumanStrategies-Csharp.ipynb)*Code inspire de Peter Norvig, 2006 - [norvig.com/sudoku.html](http://norvig.com/sudoku.html)*